In [ ]:
### International Diversification Strategy ###

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import os

# Save Results Folder
result_dir = f'results/{os.path.basename(_file_).replace(".py", "")}'
if not os.path.exists(result_dir):
    os.makedirs(result_dir)


### a. Convert stock index returns into USD usind the exchange rate

def convert_to_usd(idx_returns, cexr):
    """
    Convert stock index returns into USD using the exchange rate.
    :param idx_returns: DataFrame of stock index returns
    :param exr: DataFrame of exchange rates
    :return: DataFrame of stock index returns in USD
    """

    countries_currencies = {'AUS': 'AUD', 'CHE': 'CHF', 'DEU': 'EUR', 'FRA': 'EUR', 
                           'GBR': 'GBP', 'JPN': 'JPY'}
    
    idx_returns_usd = idx_returns.copy().drop(columns=['USA'])

    for country, currency in countries_currencies.items():
        idx_returns_usd[country] = idx_returns[country] * cexr[currency]
    
    idx_returns_usd.index = pd.to_datetime(idx_returns_usd['date'])
    idx_returns_usd.drop(columns='date', axis=1, inplace=True)

    return idx_returns_usd

### b. Compute Currency-Hedged Stock Index Returns

def compute_hedged_returns(idx_returns, cexr, ibexr):
    """
    Compute currency-hedged stock index returns.
    :param idx_returns: DataFrame of stock index returns
    :param exr: DataFrame of exchange rates
    :return: DataFrame of currency-hedged stock index returns
    """

    countries_currencies = {'AUS': 'AUD', 'CHE': 'CHF', 'DEU': 'EUR', 'FRA': 'EUR', 
                           'GBR': 'GBP', 'JPN': 'JPY'}

    S_t = cexr
    S_tp1 = cexr.shift(-1)

    S_t.set_index('date', inplace=True)
    S_tp1.set_index('date', inplace=True)

    ibexr.set_index('date', inplace=True)


    X_t = S_tp1.values/S_t.values * (1 + ibexr.drop('USD', axis=1).values) - (1 + np.tile(ibexr['USD'].values, (S_t.shape[1], 1))).T
    X_t = pd.DataFrame(X_t, columns=S_t.columns)
    X_t['date'] = pd.to_datetime(S_t.index)
    X_t.set_index('date', inplace=True)
    
    index = X_t.index.intersection(idx_returns.index)
    X_t = X_t.loc[index]
    idx_returns = idx_returns.loc[index]

    X_t = X_t[[countries_currencies[country] for country in idx_returns.columns]]
    X_t.columns = idx_returns.columns
    X_t = X_t.shift(1)
    X_t.to_csv('results/DIV/X_t.csv')

    hedged_returns = idx_returns - X_t
    hedged_returns.dropna(inplace=True)

    return hedged_returns

### c. Strategies

def compute_strategy(hedged_idx_returns, weights):
    """
    Compute the mean, standard deviation, and Sharpe ratio of the portfolio.
    :param hedged_idx_returns: DataFrame of currency-hedged stock index returns
    :param weights: List of weights for each stock index
    :return: DataFrame of portfolio returns
    """

    portfolio_returns = (hedged_idx_returns * weights.shift(1)).sum(axis=1)
    return portfolio_returns

if _name_ == "_main_":
    # Load the data
    idx_returns = pd.read_csv('data/clean/WIDX.csv')
    cexr = pd.read_csv('data/clean/CEXR.csv')
    ibexr = pd.read_csv('data/clean/IBEXR.csv')

    # Convert stock index returns into USD
    idx_returns_usd = convert_to_usd(idx_returns, cexr)
    idx_returns_usd.to_csv('results/DIV/idx_returns_usd.csv')

    # Compute currency-hedged stock index returns
    hedged_returns = compute_hedged_returns(idx_returns_usd, cexr, ibexr)
    hedged_returns.to_csv('results/DIV/hedged_returns_usd.csv')

    ### Strategies


    # a. Equal Weights
    weights = [1/hedged_returns.shape[1]] * hedged_returns.shape[1]
    weights = np.tile(weights, (hedged_returns.shape[0], 1))
    weights[:60, :] = 0  # Set first 60 months to zero for fairness in comparison
    weights = pd.DataFrame(weights, index=hedged_returns.index, columns=hedged_returns.columns)
    weights = weights.div(weights.sum(axis=1), axis=0)

    # Unhedged returns
    equal_weights_returns = compute_strategy(idx_returns_usd, weights)
    equal_weights_returns.to_csv('results/DIV/equal_weights_returns.csv')

    # Hedged returns
    equal_weights_hedged_returns = compute_strategy(hedged_returns, weights)
    equal_weights_hedged_returns.to_csv('results/DIV/equal_weights_hedged_returns.csv')


    # b. Risk Parity 60-months rolling window
    rolling_window = 60

    rolling_volatility = idx_returns_usd.rolling(window=rolling_window).std()
    rolling_volatility = pd.DataFrame(rolling_volatility, columns=idx_returns_usd.columns)
    rolling_volatility.fillna(0, inplace=True)
    risk_parity_weights = 1 / rolling_volatility
    risk_parity_weights = risk_parity_weights.div(risk_parity_weights.sum(axis=1), axis=0)
    risk_parity_returns = compute_strategy(idx_returns_usd, risk_parity_weights)
    risk_parity_returns.to_csv('results/DIV/risk_parity_returns.csv')

    rolling_volatility_hedged = hedged_returns.rolling(window=rolling_window).std()
    rolling_volatility_hedged = pd.DataFrame(rolling_volatility_hedged, columns=hedged_returns.columns)
    rolling_volatility_hedged.fillna(0, inplace=True)
    risk_parity_weights_hedged = 1 / rolling_volatility_hedged
    risk_parity_weights_hedged = risk_parity_weights_hedged.div(risk_parity_weights_hedged.sum(axis=1), axis=0)
    risk_parity_hedged_returns = compute_strategy(hedged_returns, risk_parity_weights_hedged)
    risk_parity_hedged_returns.to_csv('results/DIV/risk_parity_hedged_returns.csv')

    # c. Mean-Variance Optimal Portfolio
    def mv_optimization(cov, mean, rf, gamma=1):
        n = len(mean)

        objective = lambda w: - (np.dot(mean-rf*np.ones(n), w) - 0.5 * gamma * np.dot(w.T, np.dot(cov, w)))
        constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]

        w0 = np.ones(n) / n
        bounds = [(0, 1) for _ in range(n)]

        res = minimize(objective, w0, method='SLSQP', bounds=bounds, constraints=constraints)

        return res.x
    
    def compute_optimal_weights(returns):
        cov_matrix = returns.rolling(window=60).cov().bfill()
        mean_returns = returns.rolling(window=60).mean()
        rf = ibexr.loc[:, ['USD']]
        rf.index = pd.to_datetime(rf.index)
        rf = rf.loc[returns.index].values.reshape(-1, 1)

        optimal_weights = []

        for i in range(60):
            optimal_weights.append(0*np.ones(6))

        for i in range(len(mean_returns) - 60):
            mean = mean_returns.iloc[i+60].values
            cov = cov_matrix.iloc[i+60:i+66].values.reshape(6, 6)
            optimal_weights.append(mv_optimization(cov, mean, rf[i+60]))
        optimal_weights = pd.DataFrame(optimal_weights, index=returns.index, columns=returns.columns)
        optimal_weights = optimal_weights.div(optimal_weights.sum(axis=1), axis=0)

        return optimal_weights
    
    optimal_weights = compute_optimal_weights(idx_returns_usd)
    optimal_returns = compute_strategy(idx_returns_usd, optimal_weights)
    optimal_returns.to_csv('results/DIV/optimal_returns.csv')

    optimal_weights_hedged = compute_optimal_weights(hedged_returns)
    optimal_hedged_returns = compute_strategy(hedged_returns, optimal_weights_hedged)
    optimal_hedged_returns.to_csv('results/DIV/optimal_returns_hedged.csv')

    ### Plot the results

    plt.rcParams.update({'text.usetex': True,
                     'font.size': 16,
                     'font.family': 'serif',
                     'font.sans-serif': 'Computer Modern Roman',
                     'legend.fontsize': 16,
                     'legend.title_fontsize': 16,
                     'legend.loc': 'upper left',
                     'figure.titlesize': 16,
                     'axes.labelsize': 16,
                     'figure.figsize': (7.5, 3.5)})
    
    rf = ibexr['USD'].iloc[60:]
    rf.index = pd.to_datetime(rf.index)

    equal_weights_returns = equal_weights_returns.iloc[60:]
    risk_parity_returns = risk_parity_returns.iloc[60:]
    optimal_returns = optimal_returns.iloc[60:]
    equal_weights_hedged_returns = equal_weights_hedged_returns.iloc[60:]
    risk_parity_hedged_returns = risk_parity_hedged_returns.iloc[60:]
    optimal_hedged_returns = optimal_hedged_returns.iloc[60:]

    fig, ax = plt.subplots(1,2, figsize=(15, 7), sharey=True)

    plt.subplot(121)
    plt.plot(100*(equal_weights_returns.cumsum()), label='Equal Weights')
    plt.plot(100*(risk_parity_returns.cumsum()), label='Risk Parity')
    plt.plot(100*(optimal_returns.cumsum()), label='MV Optimal Portfolio')
    plt.plot(100*rf.cumsum(), label=r'$R_f$', color='black', linestyle='--')
    plt.xlabel('Date')
    plt.ylabel(r'Cumulative Returns [\%]')
    plt.title('Unhedged')
    plt.grid()
    plt.legend()
    plt.tight_layout()

    plt.subplot(122)
    plt.plot(100*(equal_weights_hedged_returns.cumsum()), label='Equal Weights')
    plt.plot(100*(risk_parity_hedged_returns.cumsum()), label='Risk Parity')
    plt.plot(100*(optimal_hedged_returns.cumsum()), label='MV Optimal Portfolio')
    plt.plot(100*rf.cumsum(), label=r'$R_f$', color='black', linestyle='--')
    plt.xlabel('Date')
    plt.title('Hedged')
    plt.grid()
    plt.tight_layout()
    plt.savefig('results/DIV/div_returns.pdf')
    plt.show()

    print("Mean Returns:" )
    print("Equal Weights:", 12*100*equal_weights_returns.mean())
    print("Risk Parity:", 12*100*risk_parity_returns.mean())
    print("Optimal Portfolio:", 12*100*optimal_returns.mean())
    print('\n')

    print("Std Dev: ")
    print("Equal Weights:", np.sqrt(12)*100*equal_weights_returns.std())
    print("Risk Parity:", np.sqrt(12)*100*risk_parity_returns.std())
    print("Optimal Portfolio:", np.sqrt(12)*100*optimal_returns.std())
    print('\n')

    print("Sharpe Ratio: ")
    print("Equal Weights:", (equal_weights_returns.mean() - rf.mean()) / equal_weights_returns.std())
    print("Risk Parity:", (risk_parity_returns.mean() - rf.mean()) / risk_parity_returns.std())
    print("Optimal Portfolio:", (optimal_returns.mean() - rf.mean()) / optimal_returns.std())
    print('\n')

    print("Mean Returns (Hedged):" )
    print("Equal Weights:", 12*100*equal_weights_hedged_returns.mean())
    print("Risk Parity:", 12*100*risk_parity_hedged_returns.mean())
    print("Optimal Portfolio:", 12*100*optimal_hedged_returns.mean())
    print('\n')

    print("Std Dev (Hedged): ")
    print("Equal Weights:", np.sqrt(12)*100*equal_weights_hedged_returns.std())
    print("Risk Parity:", np.sqrt(12)*100*risk_parity_hedged_returns.std())
    print("Optimal Portfolio:", np.sqrt(12)*100*optimal_hedged_returns.std())
    print('\n')

    print("Sharpe Ratio (Hedged): ")
    print("Equal Weights:", (equal_weights_hedged_returns.mean() - rf.mean()) / equal_weights_hedged_returns.std())
    print("Risk Parity:", (risk_parity_hedged_returns.mean() - rf.mean()) / risk_parity_hedged_returns.std())
    print("Optimal Portfolio:", (optimal_hedged_returns.mean() - rf.mean()) / optimal_hedged_returns.std())
    print('\n')